In [ ]:
!uv pip install -U git+https://github.com/JignyasAnand/rescript-ast-diff

In [ ]:
!uv pip install git+ssh://git@ssh.bitbucket.juspay.net/iris/jaf.git@code-traverse

In [5]:
from rescript_ast_diff import generate_pr_changes_bitbucket
from rescript_ast_diff.bitbucket import BitBucket 

BASE_URL = "https://bitbucket.juspay.net/rest"
PROJECT_KEY = "JBIZ"
REPO_SLUG = "rescript-euler-dashboard"
AUTH = ("sakthi.n@juspay.in", "BBDC-NDg5ODgwNDM2MzkyOjgy8c70YxFmjQlfjSGQD4895tx5")
HEADERS = {"Accept": "application/json"}
PR_ID = "22937"
bitbucket = BitBucket(BASE_URL, PROJECT_KEY, REPO_SLUG, AUTH, HEADERS)
generate_pr_changes_bitbucket(PR_ID, bitbucket, quiet=True)

LATEST COMMIT - 3d2100d39a1da812b6419cf590a9c23657ef3359
OLDEST COMMIT - 631110e7cd559940cbfee58c863f039bacabc74f
Changes written to -  ./detailed_changes.json


In [6]:
from jaf.core.llm import AzureGPTLLM
from dotenv import load_dotenv 
import os 

load_dotenv()

AZURE_GPT_DEPLOYMENT_NAME=os.getenv("AZURE_GPT_DEPLOYMENT_NAME")
AZURE_OAI_BASE_URL=os.getenv("AZURE_OAI_BASE_URL")
AZURE_OAI_API_VERSION= os.getenv("AZURE_OAI_API_VERSION")
AZURE_OAI_API_KEY= os.getenv("AZURE_OAI_API_KEY")

llm = AzureGPTLLM(
    deployment_name=AZURE_GPT_DEPLOYMENT_NAME,
    api_base=AZURE_OAI_BASE_URL,
    api_version=AZURE_OAI_API_VERSION,
    api_key=AZURE_OAI_API_KEY
)

In [7]:
from pydantic import BaseModel
from typing import Optional

class Data(BaseModel):
   moduleName: str
   addedFunctions: list = []
   modifiedFunctions: list = []
   deletedFunctions: list = []
   addedTypes: list = []
   modifiedTypes: list = []
   deletedTypes: list = []
   addedExternals: list = []
   modifiedExternals: list = []
   deletedExternals: list = []
   # prompt: Optional[str] = "" 
   testCases: Optional[list] = []


In [8]:
import json

with open('../python/detailed_changes.json', 'r') as file:
    data = json.load(file)

result = []
for d in data:
    result.append(Data.model_validate(d))

In [9]:
systemPrompt = """ 
You are a QA automation assistant. Given code changes to frontend components or functions (added or modified), generate structured test case objects in JSON format.

Each test case must contain:

testCaseId: Unique identifier (e.g., TC001, TC002)

description: One-line summary of what the test verifies

stepsToReproduce: A list of precise UI actions (e.g., "Click button", "Enter value in input")

expectedResult: What the correct outcome should be

⚠️ Steps must be realistic, like how a manual tester would execute them.
❌ Avoid vague steps like "Run function" — describe the UI interaction instead.

Output a JSON array of such test cases.

If there are no test cases (e.g., only deletions), return an empty list [].

Eg Format:
[
  {
    "testCaseId": "TC001",
    "description": "User can successfully search when input is valid",
    "stepsToReproduce": [
      "Navigate to the Search page",
      "Enter 'apple' in the search input box",
      "Click the 'Search' button"
    ],
    "expectedResult": "Search results are displayed for 'apple'"
  },
  {
    "testCaseId": "TC002",
    "description": "Search button does nothing if query is too short",
    "stepsToReproduce": [
      "Navigate to the Search page",
      "Enter 'ap' in the search input box",
      "Click the 'Search' button"
    ],
    "expectedResult": "No search is performed; no request sent"
  }
]
"""

userPrompt = """
Here are the frontend code changes in this pull request.
Please generate test cases in JSON format, following this structure:
[
  {{
    "testCaseId": "TC001",
    "description": "What the test is verifying",
    "stepsToReproduce": [
      "Step 1",
      "Step 2",
      "..."
    ],
    "expectedResult": "Expected behavior"
  }},
  {{
    "testCaseId": "TC002",
    "description": "What the test is verifying",
    "stepsToReproduce": [
      "Step 1",
      "Step 2",
      "..."
    ],
    "expectedResult": "Expected behavior"
  }}
  ...
]
Be realistic and user-centric in the steps — describe UI actions like clicking, typing, navigation, etc.
Generate test cases only for added or modified functions or components.
Ignore deleted functions unless they affect user-facing behavior.

Added Functions/Components:
{added}

Modified Functions/Components:
{modified}

Deleted Functions/Components:
{deleted}

Based on the above things 
Return the test cases as a JSON array.
Do not include any extra commentary or explanation — just valid JSON.
"""

In [23]:
import re
def parse(text: str) -> dict:
    cleaned = re.sub(r"```(?:json)?", "", text).strip()
    
    # Try parsing it
    try:
        parsed = json.loads(cleaned)
        return parsed
    except json.JSONDecodeError as e:
        print("❌ Failed to parse JSON:", e)
        return None

In [ ]:
import base64
import os
from google import genai
from google.genai import types


def generate(text):
    overall_response = ""
    client = genai.Client(
        api_key=os.getenv("GOOGLE_API_KEY")
    )

    model = "gemini-2.5-pro-preview-05-06"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")
        overall_response += chunk.text
    
    return overall_response



In [25]:
import tiktoken

def get_token_count(text, model="gpt-4o"):
    encoding = tiktoken.encoding_for_model(model)

    tokens = encoding.encode(text)
    return len(tokens)

In [26]:
from jaf.types import Query 

q = Query()

for res in result:
    q.prompt = userPrompt.format(added=res.addedFunctions,modified=res.modifiedFunctions,deleted=res.deletedFunctions)
    q.system_prompt = systemPrompt 

    tkn_cnt = get_token_count(q.prompt)
    print("tokenCount: ",tkn_cnt)

    if tkn_cnt >= 120000:
        print("Using Gemini")
        resp = generate(q.prompt)
        parsed_resp = parse(resp)
    else:
        print("Using Azure")
        llm(q)
        parsed_resp = parse(resp) 

    res.testCases = parsed_resp

tokenlimit:  144860
```json
[
  {
    "testCaseId": "TC001",
    "description": "Verify UI behavior for HyperPG merchant context, specifically that VPA verification might be hidden and certain order columns are adjusted.",
    "stepsToReproduce": [
      "1. Log in as a Merchant user on the HyperPG platform (e.g., hostname is 'portal.hyperpg.in').",
      "2. Navigate to a page involving VPA input or display (e.g., Payouts creation form, customer details).",
      "3. Navigate to the Orders list page.",
      "4. Attempt to download transaction reports from the Analytics section."
    ],
    "expectedResult": "The UI should reflect that VPA verification options are hidden (controlled by `hideVerifyVPA` = true). On the Orders list, the 'Preferred Gateway' column should be hidden (controlled by `orderHideCols` containing 'preferredGateway'). Downloaded transaction reports should exclude gateway-specific error/ID columns as per HyperPG merchant config."
  },
  {
    "testCaseId": "TC002",

In [13]:
json_list = [] 

for res in result:
    json_list.append(res.model_dump())

In [ ]:
with open("/cada-rescript/llm_response.json", "w") as f:
    json.dump(json_list, f, indent=4)